# Tutorial de LangChain LLM Chain

En este tutorial aprenderás a construir una aplicación sencilla de LLM con LangChain utilizando modelos de chat y plantillas de prompts. La aplicación traducirá texto de inglés a otro idioma.

## Configuración inicial

Primero, instalamos las dependencias necesarias:

In [ ]:
# Instalar las dependencias necesarias
!pip install -qU "langchain[openai]" python-dotenv

Ahora configuramos nuestras variables de entorno y API keys:

In [ ]:
import getpass
import os
from dotenv import load_dotenv

# Cargar variables desde .env si existe
load_dotenv()

# Si no está configurada la API key, solicitarla al usuario
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Introduce tu API key de OpenAI: ")

# Configurar LangSmith para trazabilidad (opcional)
if not os.environ.get("LANGSMITH_API_KEY") and input("¿Deseas configurar LangSmith para trazabilidad? (s/n): ").lower() == 's':
    os.environ["LANGSMITH_TRACING"] = "true"
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Introduce tu API key de LangSmith: ")

## 1. Utilizando Modelos de Lenguaje

Primero, inicializamos un modelo de chat de OpenAI:

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

Ahora, usaremos el modelo directamente con una lista de mensajes:

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Spanish"),
    HumanMessage("Hello, how are you?"),
]

response = model.invoke(messages)
print(response.content)

## 2. Streaming de respuestas

Los modelos de chat también soportan streaming de tokens:

In [ ]:
messages = [
    SystemMessage("Translate the following from English into French"),
    HumanMessage("Hello, how are you?"),
]

print("Streaming tokens:")
for token in model.stream(messages):
    print(token.content, end="|")

## 3. Utilizando Plantillas de Prompts

Las plantillas de prompts nos permiten estructurar nuestras entradas al modelo de forma más flexible:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

Ahora podemos formatear este template con diferentes valores:

In [ ]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})
print("Mensajes formateados:")
for message in prompt.to_messages():
    print(f"Rol: {message.type}")
    print(f"Contenido: {message.content}\n")

Finalmente, invocamos el modelo con el prompt formateado:

In [ ]:
response = model.invoke(prompt)
print(f"Traducción a italiano: {response.content}")

## 4. Aplicación interactiva de traducción

Vamos a crear una interfaz interactiva para nuestra aplicación de traducción:

In [ ]:
def translate_text():
    print("🌐 Traductor de idiomas con LangChain 🌐")
    print("----------------------------------")
    
    while True:
        text = input("\n📝 Texto en inglés (o 'salir' para terminar): ")
        if text.lower() in ["salir", "exit", "quit"]:
            print("👋 ¡Hasta pronto!")
            break
        
        language = input("🌍 Idioma de destino (e.g., Spanish, French, Italian): ")
        
        print("\n🔄 Traduciendo...")
        
        # Formatear el prompt y obtener la traducción
        prompt = prompt_template.invoke({"language": language, "text": text})
        try:
            response = model.invoke(prompt)
            print(f"\n✅ Traducción ({language}):")
            print(f"🗣️ {response.content}")
            
            # Mostrar información de tokens si está disponible
            if hasattr(response, 'response_metadata') and 'token_usage' in response.response_metadata:
                usage = response.response_metadata['token_usage']
                print(f"\nℹ️ Tokens utilizados: {usage.get('total_tokens', 'N/A')}")
        
        except Exception as e:
            print(f"\n❌ Error durante la traducción: {str(e)}")

# Descomentar la siguiente línea para ejecutar la aplicación interactiva
# translate_text()

## Conclusión

En este tutorial has aprendido:

1. Cómo usar modelos de lenguaje de chat con LangChain
2. Cómo implementar plantillas de prompts para estructurar tus entradas
3. Cómo hacer streaming de respuestas del modelo
4. Cómo construir una aplicación interactiva de traducción

Estos conceptos básicos te servirán como fundamento para construir aplicaciones más complejas utilizando LangChain.